In [1]:
import mysql.connector
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
db_user = os.environ.get('my_username')
db_password = os.environ.get('my_password')
db = mysql.connector.connect(host= "localhost", user = db_user, passwd = db_password, database = "foodie-fi")
db

# A. Customer Journey

Write a brief description about customer’s onboarding journey from result query below.

In [8]:
a1 = pd.read_sql_query("""select s.customer_id, s.plan_id, p.plan_name,p.price,s.start_date
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
where s.customer_id in (1,2,11,13,15,16,18,19) """,db)
a1

,customer_id,plan_id,plan_name,price,start_date
0,1,0,trial,0.0,2020-08-01
1,1,1,basic monthly,9.9,2020-08-08
2,2,0,trial,0.0,2020-09-20
3,2,3,pro annual,199.0,2020-09-27
4,11,0,trial,0.0,2020-11-19
5,11,4,churn,NaN,2020-11-26
6,13,0,trial,0.0,2020-12-15
7,13,1,basic monthly,9.9,2020-12-22
8,13,2,pro monthly,19.9,2021-03-29
9,15,0,trial,0.0,2020-03-17


## Findings:

Customer 1:
Started trial plan and upgraded to basic monthly plan after a week trial period ended.

Customer 15:
Started trial plan on 17th March and upgraded to basic monthly plan after a week trial period ended at 19.99. After using the plan for a month, this customer decided to terminate the overall plan.

Customer 16:
Started trial plan on 31/5 before switching to basic monthly plan for 9.99 after trial period ended. 4 months later, this customer decided to upgrade to pro annual plan that pays annually at 199.99 .

# B. Data Analysis Questions

In [9]:
# How many customers has Foodie-Fi ever had?

b1 = pd.read_sql_query("select count( distinct customer_id) as customers from subscriptions",db)
b1

,customers
0,1000


In [11]:
# What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value

b2= pd.read_sql_query("""select DATE_FORMAT(s.start_date, '%Y-%m') AS Month
from subscriptions s 
join plans p on s.plan_id = p.plan_id
where s.plan_id = 0
GROUP BY 
  Month 
ORDER BY 
  Month""",db)
b2

#FIXXXX

,Month
0,2020-01
1,2020-02
2,2020-03
3,2020-04
4,2020-05
5,2020-06
6,2020-07
7,2020-08
8,2020-09
9,2020-10


In [12]:
# What plan start_date values occur after the year 2020 for our dataset? Show the breakdown by count of events for each plan_name

b3 = pd.read_sql_query("""select p.plan_name, count(p.plan_id) as number_of_plans
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
where s.start_date >= '2021-01-01'
group by p.plan_name
order by number_of_plans""",db)
b3

,plan_name,number_of_plans
0,basic monthly,8
1,pro monthly,60
2,pro annual,63
3,churn,71


In [14]:
# What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

b4 = pd.read_sql_query("""select p.plan_id, p.plan_name, count(s.customer_id) as num_cust, round(((select count(distinct customer_id) from subscriptions where plan_id = 4)/(select count(distinct customer_id) from subscriptions))*100,1) as percentage
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
where p.plan_id = 4""",db)
b4

,plan_id,plan_name,num_cust,percentage
0,4,churn,307,30.7


In [19]:
# How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?

b5 = pd.read_sql_query("""with cte as (select s.customer_id, s.start_date, p.plan_name, s.plan_id, lead(s.plan_id) over(partition by s.customer_id) as next_plan
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
order by s.customer_id asc)
select count(*) as num_cust_churn_afterTrial, round((count(*)/(select count(distinct customer_id)from subscriptions))*100,0) as percentage_churn from cte 
where next_plan = 4 and	plan_id = 0""",db)
b5

,num_cust_churn_afterTrial,percentage_churn
0,92,9.0


In [21]:
# What is the number and percentage of customer plans after their initial free trial?

b6 = pd.read_sql_query("""with cte as (select s.customer_id, s.start_date, p.plan_name, s.plan_id, lag(s.plan_id) over(partition by s.customer_id) as prev_plan
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
order by s.customer_id asc)
select count(plan_id) as num_plan_afterChurn from cte 
where prev_plan = 0
group by plan_id
order by num_plan_afterChurn""",db)
b6

,num_plan_afterChurn
0,37
1,92
2,325
3,546


In [25]:
b8 = pd.read_sql_query("""select count(distinct s.customer_id) as num_cust_upgradeAnnual
from subscriptions s 
inner join plans p on s.plan_id = p.plan_id
where YEAR(start_date) = 2020 and s.plan_id = 3
order by s.customer_id asc""",db)
b8

,num_cust_upgradeAnnual
0,195


In [ ]:
# How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?